## Offline RL

In [1]:
# HIDDEN
import gym
import numpy as np

In [2]:
# HIDDEN
import ray
import logging
ray.init(log_to_driver=False, ignore_reinit_error=True, logging_level=logging.ERROR); # logging.FATAL

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

#### Ist das realistisch?

- Bis jetzt haben wir eine Simulation des Nutzerverhaltens erstellt
- Bei einigen Anwendungen können wir vielleicht genaue Simulationen erstellen:
  - physiksimulationen (z. B. Roboter)
  - spiele
  - wirtschaftliche/finanzielle Simulationen?
- Für das Nutzerverhalten ist dies jedoch schwierig

#### Ist das realistisch? 

- Am besten wäre es, RL live einzusetzen, aber das ist nicht praktikabel
- Eine andere Möglichkeit: aus Nutzerdaten lernen?
- Wir können das mit **offline reinforcement learning** machen

#### Offline RL

- Was ist Offline-RL?
- Erinnere dich an unsere RL-Schleife:

![](img/RL-loop-3.png)

#### Offline RL

- Im Offline-RL haben wir keine Environment, mit der wir in einer Feedbackschleife interagieren können:

![](img/offline-RL-loop.png)

Diese historischen Daten wurden durch eine andere, unbekannte Policy/andere Policyen erzeugt.

Anmerkungen:

Könnten von echten Nutzern oder von einer anderen Quelle (zufällig oder von einem RL-Agenten!) erzeugt worden sein

#### Herausforderung des Offline-RL

- Wir können keine "Was wäre wenn"-Fragen beantworten
- Wir können nur die Ergebnisse der versuchten Aktionen im Datensatz sehen

Anmerkungen:

Vielleicht lernen wir dadurch zu schätzen, wie wertvoll und toll es ist, eine Simulation zur Verfügung zu haben, die wir den ganzen Kurs über hatten. Sie ermöglicht es uns, alles auszuprobieren, ohne dass es uns etwas kostet, außer den Rechenaufwand (vorausgesetzt, es handelt sich um einen Simulator und nicht um eine reale Environment) 

In [3]:
# # HIDDEN

# # generate the offline dataset
# env_config = {
#     "num_candidates" : 2,
#     "alpha"          : 0.5,
#     "seed"           : 42
# }

# from ray.rllib.algorithms.ppo import PPO, PPOConfig

# ppo_config = (
#     PPOConfig()\
#     .framework("torch")\
#     # need to set num_rollout_workers=1 for now per https://github.com/ray-project/ray/issues/25696
#     .rollouts(create_env_on_local_worker=True, num_rollout_workers=1)\
#     .debugging(seed=0, log_level="ERROR")\
#     .training(model={"fcnet_hiddens" : [64, 64]}, lr=0.001)\
#     # .environment(env_config=env_config)\
#     .offline_data(output="data/recommender2")
# )

# from envs_04 import BasicRecommenderWithHistory

# ppo_history = ppo_config.build(env="CartPole-v1")

# rewards_history = []
# for i in range(25):
#     result = ppo_history.train()
#     rewards_history.append(result["episode_reward_mean"])
    
# ppo_history.evaluate(duration_fn=1000)["evaluation"]["episode_reward_mean"]

#### Empfehlungsdatensatz

- Schauen wir uns einen Offline-Datensatz an, aus dem wir lernen können.
- Wir brauchen ein bisschen Code, um alle JSON-Objekte in der Datei zu lesen:

In [4]:
import json

json_dataset_file = "data/recommender_offline.json"

rollouts = []
with open(json_dataset_file, "r") as f:
    for line in f:
        data = json.loads(line)
        rollouts.append(data)

In [5]:
len(rollouts)

50

Wir haben 50 "Rollouts" von Daten.

Anmerkungen:

Die Datei ist in dem Format, aus dem RLlib lernt.

#### Empfehlungsdatensatz

Jedes Rollout ist ein Diktat mit Informationen über den Zeitschritt:

In [6]:
from ray.rllib.utils.compression import unpack, pack

obs = unpack(rollouts[0]["obs"])
obs.shape

(200, 2)

- Wir haben 200 Zeitschritte an Daten in jedem Rollout
- Schauen wir uns zuerst die Beobachtungen an

Anmerkungen:

Die Zahl 200 wird durch den Konfigurationsparameter "rollout_fragment_length" des Algorithmus festgelegt.

#### Empfehlungsdatensatz

Hier sind die ersten 3 Beobachtungen:

In [7]:
obs[:3]

array([[0.6545137 , 0.29728338],
       [0.5238871 , 0.5144319 ],
       [0.6741674 , 0.10163702]], dtype=float32)


Wir können sehen, dass "Anzahl_Kandidaten" auf 2 gesetzt wurde

#### Empfehlungsdatensatz

Wir können uns auch die ersten 3 Aktionen, Belohnungen und Spenden ansehen:

In [8]:
rollouts[0]["actions"][:3]

[0, 0, 1]

In [9]:
rollouts[0]["rewards"][:3]

[0.6545137166976929, 0.3524414300918579, 0.05838315561413765]

In [10]:
rollouts[0]["dones"][:3]

[False, False, False]

Anmerkungen:

Zuerst sah der Agent also die Beobachtung [0,65, 0,297] aus der vorherigen Folie, dann führte er die Aktion 0 aus, erhielt eine Belohnung von 0,65 und die Episode wurde nicht ausgeführt.

Der Datensatz enthält noch mehr Informationen als die oben genannten, aber das sind die wichtigsten Punkte.

#### Überwachtes Lernen

- Moment, das ist ein Datensatz... können wir nicht einfach überwachtes Lernen anwenden, um Zustände auf Aktionen abzubilden? 🤔
- Ja, das können wir, und das würde darauf abzielen, die Policy, die den Datensatz erzeugt hat, wiederherzustellen (_Nachahmungslernen_).
- Aber es ist tatsächlich möglich, etwas _Besseres_ zu tun ... und das ist unser Ziel mit Offline-RL.

In [11]:
# TODO
# maybe going overboard here, but for a synthetic example we could actually show it does better than the policy that generated the data,
# since we generated the data ourselves

#### Offline-RL-Training

- Viele Infos über Offline-RL mit RLlib [hier](https://docs.ray.io/en/latest/rllib/rllib-offline.html).
- Zuerst brauchen wir einen Algorithmus.
- Für Offline-RL können wir `PPO` nicht verwenden.
- Wir werden den Algorithmus "MARWIL" verwenden, der in der RLlib enthalten ist.

In [12]:
from ray.rllib.algorithms.marwil import MARWIL, MARWILConfig

#### Offline-RL-Training

Als Nächstes erstellen wir die Konfiguration, wobei wir mit `MARWILConfig` statt mit `PPOConfig` beginnen

In [13]:
# This is the same as before
offline_config = ( 
    MARWILConfig()\
    .framework("torch")\
    .rollouts(create_env_on_local_worker=True)\
    .debugging(seed=0, log_level="ERROR")\
    .training(model={"fcnet_hiddens" : [64, 64]})\
)

# This is new for offline RL
num_candidates = 2
offline_config = offline_config.environment(
    observation_space = gym.spaces.Box(low=0, high=1, shape=(num_candidates,)), 
    action_space = gym.spaces.Discrete(num_candidates),
).offline_data(
    input_ = [json_dataset_file],
)

Anmerkungen:

- Die Konfigurationselemente auf der Oberseite sollten dir bekannt vorkommen. In der zweiten Hälfte sind die Dinge ein bisschen anders:
  - Wir müssen den Pfad zur Dataset-Datei angeben
  - Da es keine env gibt, müssen wir die Beobachtungs- und Aktionsräume manuell angeben
- Wir haben keine Environmentskonfiguration, weil es keine Environment gibt!

#### Ausbildung

In [14]:
marwil = offline_config.build()

In [15]:
for i in range(50):
    marwil.train()

#### Bewertung

wie können wir ohne einen Simulator evaluieren?

- Das nennt man Off-Policy-Schätzung.
- Das ist eine kompliziertere Technik und liegt außerhalb des Rahmens dieses Kurses.
- Siehe die RLlib-Dokumente [hier] (https://docs.ray.io/en/latest/rllib/rllib-offline.html).
- Wir werden die Auswertung mit unserem Simulator durchführen, da wir ihn ja haben.

Anmerkungen:

In einem realistischen Offline-RL-Szenario hast du keinen Simulator zur Verfügung. Dann musst du komplexere Evaluierungstechniken anwenden, die Off-Policy-Schätzung genannt werden. Du hättest Trainings- und Testdatensätze wie beim überwachten Lernen 

Da wir aber einen Simulator haben, werden wir ihn für die Auswertung verwenden. RLlib hat eine Option, die das erlaubt, denn es ist nützlich für die Fehlersuche und so weiter. Wir sehen uns das auf der nächsten Folie an.

#### Bewertung mit unserem Simulator

Wir können den Algorithmus mit unserem env-Simulator bewerten.

In [16]:
# HIDDEN
from envs_04 import BasicRecommender

env_config = {
    "num_candidates" : 2,
    "alpha"          : 0.5,
    "seed"           : 42
}

In [17]:
env = BasicRecommender(env_config)

def get_episode_reward(env, algo):
    obs = env.reset()
    done = False
    total_reward = 0
    while not done:
        action = algo.compute_single_action(obs)
        obs, reward, done, _ = env.step(action)
        total_reward += reward
    return total_reward

- Oben: Richte eine Funktion ein, die eine Episode mit dem Simulator abspielt.
- Unten: lasse dies für 100 Episoden laufen, um den Mittelwert zu erhalten.

In [18]:
np.mean([get_episode_reward(env, marwil) for i in range(100)])

25.537665635536726

Das scheint wieder in etwa so zu sein wie beim Zufallsprinzip (25,5).

#### RLlib für simulatorbasierte Bewertung

RLlib bietet dies ebenfalls als Funktion an:

In [31]:
offline_config = offline_config.evaluation(
    off_policy_estimation_methods={},
    evaluation_config={
        "input": "sampler",
        "env": BasicRecommender,
        "env_config" : env_config
    }
)

In [ ]:
# HIDDEN
#marwil.stop()

Wenn wir die Konfiguration so eingerichtet und trainiert hätten:

In [32]:
marwil = offline_config.build()

In [33]:
for i in range(10):
    marwil.train()

In [30]:
marwil.evaluate()["evaluation"]["episode_reward_mean"]

25.3457144503546

Auch hier sehen wir ein ähnliches Ergebnis.

#### Lass uns das Gelernte anwenden!

## Beispiel für offline RL
<!-- multiple choice -->

- [Einer KI das Schachspielen beibringen, indem man sie wiederholt gegen andere KIs spielen lässt.
- [x] Einer KI das Schachspielen beibringen, indem man sie vergangene Partien von professionellen Schachspielern spielen lässt.

## Helfen Offline-Daten?
<!-- multiple choice -->

Stell dir vor, du hast eine Environment, die deine Environment perfekt abbildet; zum Beispiel trainierst du eine KI für ein Einzelspielerspiel wie [Atari Breakout](https://en.wikipedia.org/wiki/Breakout_(video_game)). Zusätzlich zum Simulator hast du auch einige Offline-Daten zur Verfügung. Auch wenn es in den obigen Folien nicht erwähnt wird, ist es möglich, einen Simulator mit Offline-Daten zu kombinieren, um gemeinsam einen Agenten mit RL zu trainieren (siehe [hier](https://docs.ray.io/en/latest/rllib/rllib-offline.html#mixing-simulation-and-offline-data)). Wenn du bereits über einen perfekten Simulator verfügst, könnten Offline-Daten einen zusätzlichen Nutzen bringen, wenn sie beim Training mit dem Simulator kombiniert werden?

#### Beste Policy nach dem Training

Wähle die richtige Aussage zum Finden der besten Policy nach einem unbegrenzten Training. Du kannst davon ausgehen, dass du einen "perfekten" RL-Algorithmus hast, d.h. er kann jede Policy darstellen und jede Funktion optimieren.

- [x] Mit einem "perfekten" RL-Algorithmus und genügend Rechenzeit bringt es keinen Vorteil, die historischen Daten zu verwenden, weil der Simulator alles enthält, was man über die Environment wissen muss.
- [Bei einem "perfekten" RL-Algorithmus und unbegrenzter Rechenzeit können dir die historischen Daten helfen, eine bessere Policy zu finden als nur mit dem Simulator. | Mit einem "perfekten" RL-Algorithmus solltest du irgendwann die optimale Policy finden. Das ist so, als hätte man unendlich viele Daten, unendlich viel Rechenzeit, ein beliebig komplexes Modell und eine perfekte Optimierungsmethode.

#### Trainingsgeschwindigkeit

Wähle die richtige Aussage über das Finden der besten Policy nach ein wenig Training. Du kannst davon ausgehen, dass du einen "perfekten" RL-Algorithmus hast, d.h. er kann jede Policy darstellen und jede Funktion optimieren.

- [Wenn du einen "perfekten" RL-Algorithmus hast, aber nur ein bisschen Rechenzeit, bringt es nichts, die historischen Daten zu verwenden, weil der Simulator alles enthält, was man über die Environment wissen muss. | Was wäre, wenn die historischen Daten mit der *optimalen* Policy erzeugt worden wären?
- [Mit einem "perfekten" RL-Algorithmus, aber nur wenig Rechenzeit, können dir die historischen Daten helfen, eine bessere Policy zu finden als nur mit dem Simulator. | Wenn die historischen Daten mit einer sehr guten Policy erzeugt wurden, könntest du schnell daraus lernen.

## Historische Datenpolicy
<!-- multiple choice -->

Offline-RL basiert auf Daten, die von einer Policy erzeugt werden, die mit der Environment interagiert. Welche der folgenden Eigenschaften ist **NICHT** eine wünschenswerte Eigenschaft dieses Datensatzes / dieser historischen Policy?
 
- [x] Die Environment und die historische Policy sind beide deterministisch. | In diesem Fall würden wir nur eine Trajektorie durch die Environment erkunden 
- [ ] Der Datensatz enthält eine große Anzahl von Episoden.
- [Die historische Policy erkundet eine Vielzahl von Zuständen in der Environment.
- [Die historische Policy erzielt in einigen Episoden eine hohe Belohnung.   

## Offline RL für Cartpole
<!-- coding exercise -->

In dieser Übung werden wir das berühmte [Cartpole-Benchmark-Problem] (https://www.gymlibrary.ml/environments/classic_control/cart_pole/) angehen, das mit der `gym`-Bibliothek ausgeliefert wird. Das Ziel ist es, das umgedrehte Pendel vor dem Umfallen zu bewahren, indem wir bei jedem Zeitschritt eine Kraft nach links oder rechts ausüben 

Wir trainieren den Agenten mit Offline-Daten, die in der Datei `cartpolev1_offline.json` enthalten sind, die vom Code eingelesen wird.

Wir werden den Agenten mit dem echten Cartpole-Simulator _auswerten_. (Auch hier gilt: Wenn wir Offline-RL verwenden würden, hätten wir wahrscheinlich keinen Zugriff auf den echten Simulator, aber wir fügen ihn hier ein, damit wir unseren Agenten auf der Basis von Fakten bewerten können)

Fülle den fehlenden Code aus. Führe dann den Code aus und beantworte die Multiple-Choice-Frage unten.

In [ ]:
# EXERCISE
import gym
import numpy as np
import matplotlib.pyplot as plt

from ray.rllib.algorithms.marwil import MARWIL, MARWILConfig

offline_trainer_config = {
    # These should look familiar:
    "framework"             : "torch",
    "create_env_on_driver"  : True,
    "seed"                  : 0,
    "model"                 : {
        "fcnet_hiddens"     : [64, 64]
    },
    
    # These are new for offline RL:
    ____: ["data/cartpolev1_offline.json"],
    "observation_space": gym.spaces.Box(low=____, 
                                        high=np.array([4.8,  np.inf,  0.42,  np.inf])),
    "action_space": gym.spaces.Discrete(2),
    "input_evaluation" : ["simulation", "is", "wis"],
    "env" : "CartPole-v1" # for evaluation only
}

algo = ...

# Training (and storing results)
results_off = []
results_sim = []
for i in range(200):
    r = algo.____()
    results_off.append(r["off_policy_estimator"]["wis"]['V_gain_est'])
    results_sim.append(r["episode_reward_mean"])

plt.plot(results_sim);
plt.xlabel('iterations') 
plt.ylabel('simulator reward') 

In [30]:
# SOLUTION
import gym
import numpy as np
import matplotlib.pyplot as plt
from ray.rllib.algorithms.marwil import MARWIL, MARWILConfig
from ray.rllib.algorithms.crr import CRR, CRRConfig

# This is the same as before
offline_config = ( 
    CRRConfig()\
    .framework("torch")\
    .rollouts(create_env_on_local_worker=True)\
    .debugging(seed=0, log_level="ERROR")\
    .training(model={"fcnet_hiddens" : [32, 32]})\
)
# This is new for offline RL
offline_config = offline_config.environment(
    observation_space = gym.spaces.Box(low=np.array([-4.8, -np.inf, -0.42, -np.inf]), 
                                        high=np.array([4.8,  np.inf,  0.42,  np.inf])),
    action_space = gym.spaces.Discrete(2),
    env = "CartPole-v1" # for evaluation only
).offline_data(
    input_ = ["data/cartpolev1_offline.json"]
).evaluation(
    off_policy_estimation_methods={'simulation': {'type': 'simulation'}}
)

algo = offline_config.build()

# Training (and storing results)
results_off = []
results_sim = []
for i in range(10):
    print(i)
    r = algo.train()
    results_off.append(r[""]["wis"]['v_new_mean'])
    results_sim.append(r["episode_reward_mean"])

plt.plot(results_sim);
plt.xlabel('iterations') 
plt.ylabel('simulator reward') 

ValueError: not enough values to unpack (expected 2, got 1)

In [31]:
# TODO
# see the tuned examples here: https://docs.ray.io/en/master/rllib/rllib-algorithms.html#offline

In [ ]:
# TODO:
# - ALSO: this might be a great example to try supervised learning and show why it doesn't work...???
#   - in some ways this is a way better example than frozen lake... because there IS a short-term reward, it's just not what you should look at.
#   - with frozen lake there is no short-term reward, so RL seems "obvious"
# Or, that could go in the offline RL section, since we already have a data file there and could do SL directly on it
# Yes, that seems cool.
# could use scipy/numpy to just do the normal equations if we want to avoid adding a dependency on sklearn 

# Supervised learning only gives you imitation learning, which can only get as good as the policy that generated the data